In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x2ef3910
Welcome to JupyROOT 6.30/06


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(0)
# for nicer 2d colz
ROOT.ildStyle.SetPalette(ROOT.kBird)
ROOT.ildStyle.SetNumberContours(256)

In [5]:
%%cpp
using namespace ROOT::VecOps;
using namespace ROOT::Math::VectorUtil;

In [6]:
prefix = "data/gen/simple_whizard"
df = ROOT.RDataFrame("events", (f"{prefix}/enuqq.edm4hep.root", f"{prefix}/munuqq.edm4hep.root"))
# df = ROOT.RDataFrame("events", "data/truejet/test/sw_sl/nano.root")
# df = ROOT.RDataFrame("events", "data/truejet/test/ww_sl/nano.root")
df.Describe()

Dataframe from TChain events in files
  data/gen/simple_whizard/enuqq.edm4hep.root
  data/gen/simple_whizard/munuqq.edm4hep.root

Property                Value
--------                -----
Columns in total           67
Columns from defines        0
Event loops run             0
Processing slots            6

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeader                             ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeader.eventNumber                 ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.runNumber                   ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.timeStamp                   ROOT::VecOps::RVec<ULong_t>                     Dataset
EventHeader.weight                      ROOT::VecOps::RVec<Double_t>                    Dataset
EventHeader.weights

In [7]:
# these files are very simple, they have the Ws explicitly and there is only one stable e/p
df = df.Define("l_MC_idx", "ArgMax((abs(MCParticles.PDG) == 11 || abs(MCParticles.PDG) == 13) && MCParticles.generatorStatus == 1)")
df = df.Define("nu_MC_idx", "ArgMax((abs(MCParticles.PDG) == 12 || abs(MCParticles.PDG) == 14) && MCParticles.generatorStatus == 1)")

df = df.Define("l_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[l_MC_idx], MCParticles.momentum.y[l_MC_idx], MCParticles.momentum.z[l_MC_idx], MCParticles.mass[l_MC_idx])")
df = df.Define("nu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[nu_MC_idx], MCParticles.momentum.y[nu_MC_idx], MCParticles.momentum.z[nu_MC_idx], MCParticles.mass[nu_MC_idx])")

df = df.Define("l_pdg", "MCParticles.PDG[l_MC_idx]")
df = df.Define("nu_pdg", "MCParticles.PDG[nu_MC_idx]")

In [8]:

df = df.Define("l_MC_lvec_e", "l_MC_lvec.energy()")
df = df.Define("l_MC_lvec_theta", "l_MC_lvec.theta()")
df = df.Define("l_MC_lvec_cosTheta", "cos(l_MC_lvec_theta)")

df = df.Define("l_MC_lvec_pt", "l_MC_lvec.pt()")
df = df.Define("l_MC_lvec_pz", "l_MC_lvec.pz()")

df = df.Define("nu_MC_lvec_e", "nu_MC_lvec.energy()")

df = df.Define("lnu_MC_m", "(l_MC_lvec + nu_MC_lvec).mass()")
df = df.Define("lnu_MC_esum", "l_MC_lvec_e + nu_MC_lvec_e")

# FIXME: no crossing angle!
df = df.Define("l_beam_lvec", "ROOT::Math::PxPyPzMVector(0, 0, std::copysign(125., l_pdg), 0.0005109968178652498)")
df = df.Define("q2", "(l_beam_lvec - nu_MC_lvec).M2()")

df = df.Define("lnu_angle", "ROOT::Math::VectorUtil::Angle(l_MC_lvec, nu_MC_lvec)")

In [9]:
df = df.Define("weight", "1.0")

In [10]:
df = df.Filter("abs(l_MC_lvec_cosTheta) < cos(0.150)")

In [13]:
h_l_e = df.Histo1D(("", ";E_{e} [GeV]", 300, 0., 150.), "l_MC_lvec_e", "weight")
h_nu_e = df.Histo1D(("", ";E_{#nu} [GeV]", 300, 0., 150.), "nu_MC_lvec_e", "weight")

h_2d_lnu_e = df.Histo2D(("", ";E_{e} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "l_MC_lvec_e", "nu_MC_lvec_e", "weight")
h_2d_enu_e = df.Filter("abs(l_pdg) == 11").Histo2D(("", ";E_{e} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "l_MC_lvec_e", "nu_MC_lvec_e", "weight")
h_2d_munu_e = df.Filter("abs(l_pdg) == 13").Histo2D(("", ";E_{e} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "l_MC_lvec_e", "nu_MC_lvec_e", "weight")

h_enu_esum = df.Filter("abs(l_pdg) == 11").Histo1D(("", "e#nuqq", 100, 50., 200.), "lnu_MC_esum", "weight")
h_munu_esum = df.Filter("abs(l_pdg) == 13").Histo1D(("", "mu#nuqq", 100, 50., 200.), "lnu_MC_esum", "weight")

h_enu_m = df.Filter("abs(l_pdg) == 11").Histo1D(("", "e#nuqq;m_{l+#nu} [GeV]", 50, 0., 200.), "lnu_MC_m", "weight")
h_munu_m = df.Filter("abs(l_pdg) == 14").Histo1D(("", "#mu#nuqq;m_{l+#nu} [GeV]", 50, 0., 200.), "lnu_MC_m", "weight")

h_q2 = df.Histo1D("q2")
h_enu_q2 = df.Filter("abs(l_pdg) == 11").Histo1D(("", "e#nuqq;q^{2} [GeV^{2}]", 35, -60e3, 1e3), "q2", "weight")
h_munu_q2 = df.Filter("abs(l_pdg) == 13").Histo1D(("", "#mu#nuqq;q^{2} [GeV^{2}]", 35, -60e3, 1e3), "q2", "weight")

h_2d_q2_m = df.Histo2D(("", ";q^{2} [GeV^{2}];m_{l,#nu} [GeV]", 70, -60e3, 1e3, 100, 0., 200.), "q2", "lnu_MC_m", "weight")
h_2d_enu_q2_m = df.Filter("abs(l_pdg) == 11").Histo2D(("", ";q^{2} [GeV^{2}];m_{l+#nu} [GeV]", 70, -60e3, 1e3, 100, 0., 200.), "q2", "lnu_MC_m", "weight")
h_2d_munu_q2_m = df.Filter("abs(l_pdg) == 13").Histo2D(("", ";q^{2} [GeV^{2}];m_{l+#nu} [GeV]", 70, -60e3, 1e3, 100, 0., 200.), "q2", "lnu_MC_m", "weight")

h_2d_enu_esum_m = df.Filter("abs(l_pdg) == 11").Histo2D(("", ";E_{l}+ E_{#nu} [GeV];m_{l+#nu} [GeV]", 100, 0., 200., 100, 0., 200.), "lnu_MC_esum", "lnu_MC_m", "weight")
h_2d_munu_esum_m = df.Filter("abs(l_pdg) == 13").Histo2D(("", ";E_{l}+ E_{#nu} [GeV];m_{l+#nu} [GeV]", 100, 0., 200., 100, 0., 200.), "lnu_MC_esum", "lnu_MC_m", "weight")

h_2d_enu_q2_esum = df.Filter("abs(l_pdg) == 11").Histo2D(("", ";q^{2} [GeV^{2}];E_{l}+ E_{#nu} [GeV]", 70, -60e3, 1e3, 100, 0., 200.), "q2", "lnu_MC_esum", "weight")
h_2d_munu_q2_esum = df.Filter("abs(l_pdg) == 13").Histo2D(("", ";q^{2} [GeV^{2}];E_{l}+ E_{#nu} [GeV]", 70, -60e3, 1e3, 100, 0., 200.), "q2", "lnu_MC_esum", "weight")

h_e_theta = df.Filter("abs(l_pdg) == 11").Histo1D(("", ";l #theta", 100, 0., 3.2), "l_MC_lvec_theta", "weight")
h_mu_theta = df.Filter("abs(l_pdg) == 13").Histo1D(("", ";l #theta", 100, 0., 3.2), "l_MC_lvec_theta", "weight")

h_e_lnu_angle = df.Filter("abs(l_pdg) == 11").Histo1D(("", ";angle(l, nu)", 100, 0., 3.2), "lnu_angle", "weight")
h_mu_lnu_angle = df.Filter("abs(l_pdg) == 13").Histo1D(("", ";angle(l, nu)", 100, 0., 3.2), "lnu_angle", "weight")

In [14]:
plots = "plots/signal_region_old"


c_l_e = ROOT.TCanvas()
h_l_e.Draw()
c_l_e.Draw()

c_nu_e = ROOT.TCanvas()
h_nu_e.Draw()
c_nu_e.Draw()

c_2d_enu_e = ROOT.TCanvas()
h_2d_enu_e.Draw("colz0")
c_2d_enu_e.SetLogz()
c_2d_enu_e.Draw()

c_2d_munu_e = ROOT.TCanvas()
h_2d_munu_e.Draw("colz0")
c_2d_munu_e.SetLogz()
c_2d_munu_e.Draw()

c_lnu_esum = ROOT.TCanvas()
s_lnu_esum = ROOT.THStack("", ";E_{l}+ E_{#nu} [GeV]")
s_lnu_esum.Add(h_munu_esum.GetPtr())
h_enu_esum.SetMarkerColor(ROOT.kRed)
h_enu_esum.SetLineColor(ROOT.kRed)
s_lnu_esum.Add(h_enu_esum.GetPtr())
s_lnu_esum.Draw("nostack hist")
l_lnu_esum = c_lnu_esum.BuildLegend(0.525, 0.75, 0.75, 0.9)
c_lnu_esum.Draw()
c_lnu_esum.SaveAs(f"{plots}/lnu_esum.pdf")

c_lnu_m_sw = ROOT.TCanvas()
s_lnu_m_sw = ROOT.THStack("", ";m_{l+#nu} [GeV]")
#h_munu_m.Draw("hist")
s_lnu_m_sw.Add(h_munu_m.GetPtr())
h_enu_m.SetMarkerColor(ROOT.kRed)
h_enu_m.SetLineColor(ROOT.kRed)
#h_enu_m.Draw("same hist")
s_lnu_m_sw.Add(h_enu_m.GetPtr())
s_lnu_m_sw.Draw("nostack hist")
l_lnu_m_sw = c_lnu_m_sw.BuildLegend(0.5, 0.75, 0.75, 0.9)
c_lnu_m_sw.Draw()
c_lnu_m_sw.SaveAs(f"{plots}/lnu_m.pdf")

c_lnu_m_sw_log = ROOT.TCanvas()
s_lnu_m_sw_log = ROOT.THStack("", ";m_{l+#nu} [GeV]")
s_lnu_m_sw_log.Add(h_munu_m.GetPtr())
s_lnu_m_sw_log.Add(h_enu_m.GetPtr())
s_lnu_m_sw_log.Draw("nostack hist")
l_lnu_m_sw_log = c_lnu_m_sw_log.BuildLegend(0.5, 0.75, 0.75, 0.9)
c_lnu_m_sw_log.SetLogy()
c_lnu_m_sw_log.Draw()
c_lnu_m_sw_log.SaveAs(f"{plots}/lnu_m_log.pdf")

c_lnu_q2_sw = ROOT.TCanvas()
s_lnu_q2_sw = ROOT.THStack("", ";q^{2} [GeV^{2}]")
#h_munu_q2.Draw("hist")
s_lnu_q2_sw.Add(h_munu_q2.GetPtr())
h_enu_q2.SetMarkerColor(ROOT.kRed)
h_enu_q2.SetLineColor(ROOT.kRed)
#h_enu_q2.Draw("same hist")
s_lnu_q2_sw.Add(h_enu_q2.GetPtr())
s_lnu_q2_sw.Draw("nostack hist")
l_lnu_q2_sw = c_lnu_q2_sw.BuildLegend(0.2, 0.775, 0.5, 0.9)
c_lnu_q2_sw.Draw()
c_lnu_q2_sw.SaveAs(f"{plots}/lnu_q2.pdf")

c_lnu_q2_sw_log = ROOT.TCanvas()
s_lnu_q2_sw_log = ROOT.THStack("", ";q^{2} [GeV^{2}]")
s_lnu_q2_sw_log.Add(h_munu_q2.GetPtr())
s_lnu_q2_sw_log.Add(h_enu_q2.GetPtr())
s_lnu_q2_sw_log.Draw("nostack hist")
l_lnu_q2_sw_log = c_lnu_q2_sw_log.BuildLegend(0.2, 0.775, 0.5, 0.9)
c_lnu_q2_sw_log.SetLogy()
c_lnu_q2_sw_log.Draw()
c_lnu_q2_sw_log.SaveAs(f"{plots}/lnu_q2_log.pdf")

# c_2d_q2_m = ROOT.TCanvas()
# h_2d_q2_m.Draw("colz0")
# c_2d_q2_m.SetLogz()
# c_2d_q2_m.Draw()
# c_2d_q2_m.SaveAs(f"{plots}/2d_q2_m_log.pdf")

c_2d_enu_q2_m_log = ROOT.TCanvas()
h_2d_enu_q2_m.Draw("colz0")
c_2d_enu_q2_m_log.SetLogz()
c_2d_enu_q2_m_log.Draw()
c_2d_enu_q2_m_log.SaveAs(f"{plots}/2d_enu_q2_m_log.pdf")
# c_2d_enu_q2_m = ROOT.TCanvas()
# h_2d_enu_q2_m.Draw("colz0")
# c_2d_enu_q2_m.Draw()
# c_2d_enu_q2_m.SaveAs(f"{plots}/2d_enu_q2_m.pdf")

c_2d_munu_q2_m_log = ROOT.TCanvas()
h_2d_munu_q2_m.Draw("colz0")
c_2d_munu_q2_m_log.SetLogz()
c_2d_munu_q2_m_log.Draw()
c_2d_munu_q2_m_log.SaveAs(f"{plots}/2d_munu_q2_m_log.pdf")

c_2d_enu_esum_m_log = ROOT.TCanvas()
h_2d_enu_esum_m.Draw("colz0")
c_2d_enu_esum_m_log.SetLogz()
c_2d_enu_esum_m_log.Draw()
c_2d_enu_esum_m_log.SaveAs(f"{plots}/2d_enu_esum_m_log.pdf")

c_2d_munu_esum_m_log = ROOT.TCanvas()
h_2d_munu_esum_m.Draw("colz0")
c_2d_munu_esum_m_log.SetLogz()
c_2d_munu_esum_m_log.Draw()
c_2d_munu_esum_m_log.SaveAs(f"{plots}/2d_munu_esum_m_log.pdf")

c_2d_enu_q2_esum_log = ROOT.TCanvas()
h_2d_enu_q2_esum.Draw("colz0")
c_2d_enu_q2_esum_log.SetLogz()
c_2d_enu_q2_esum_log.Draw()
c_2d_enu_q2_esum_log.SaveAs(f"{plots}/2d_enu_q2_esum_log.pdf")

c_2d_munu_q2_esum_log = ROOT.TCanvas()
h_2d_munu_q2_esum.Draw("colz0")
c_2d_munu_q2_esum_log.SetLogz()
c_2d_munu_q2_esum_log.Draw()
c_2d_munu_q2_esum_log.SaveAs(f"{plots}/2d_enu_q2_musum_log.pdf")

c_l_theta = ROOT.TCanvas()
h_e_theta.Draw("e1")
h_mu_theta.Draw("same hist")
c_l_theta.Draw()

c_lnu_angle = ROOT.TCanvas()
h_mu_lnu_angle.Draw("hist")
h_e_lnu_angle.Draw("same e1")
c_lnu_angle.Draw()

Info in <TCanvas::Print>: pdf file plots/signal_region_old/lnu_esum.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/lnu_m.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/lnu_m_log.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/lnu_q2.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/lnu_q2_log.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/2d_enu_q2_m_log.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/2d_munu_q2_m_log.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/2d_enu_esum_m_log.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/2d_munu_esum_m_log.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/2d_enu_q2_esum_log.pdf has been created
Info in <TCanvas::Print>: pdf file plots/signal_region_old/2d_enu_q2